# Librerias

In [1]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from sqlalchemy import create_engine
import json
import psycopg2
from psycopg2.extras import execute_batch
import numpy as np
import re
import unicodedata
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Funciones

## Conexion a Base de datos

In [2]:
def conection_bd():
    try:
        host = "directob2b.postgres.database.azure.com"
        port = "5432"
        database = "ecommerce"
        user = "directob2b"
        password = "Noctua2025$"
        engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
        print("Conexión exitosa 🚀")
        return engine
    except Exception as e:
        print("Error al conectar con SQLAlchemy:", e)
        return None

## Datos Produtos Arbol

In [3]:
engine = conection_bd()
try:
    query = """SELECT tp.csku,
       tp.cnombre,
       tp.cdescripcion,
       tp.cmarca,
       tnc.id_subcategoria as id_subcategoria,
       tnc.nombre_subcategoria as subcategoria
FROM tbl_producto as tp
  LEFT JOIN tbl_nueva_subcategoria as tnc
    ON tnc.id_subcategoria = tp.id_nueva_subcategoria"""
    df_productos = pd.read_sql(query, engine)
    print('Catálogo obtenido')
except Exception as e:
    print("Error al ejecutar la consulta:", e)


engine.dispose()

Conexión exitosa 🚀
Catálogo obtenido


## Ver prodcutos

In [4]:
df_productos.sample(5)

,csku,cnombre,cdescripcion,cmarca,id_subcategoria,subcategoria
576,4282385,TAPA PARA 884,TAPA PARA 884,Faac,NaN,None
69814,WDS100T2XHE-00BCA0,UNIDAD DE ESTADO SOLIDO SSD INTERNO WD BLACK S...,UNIDAD DE ESTADO SOLIDO SSD INTERNO WD BLACK S...,Western Digital,3.0,Componentes Informáticos
14016,12U2000YLS,PCs de Escritorio LENOVO 12U2000YLS,PC de escritorio LENOVO ThinkCentre M70S Gen 5...,Lenovo Idea,5.0,Computadores
32582,70330331952,Marcador bic permanente intensity gri,"Marcador permanente de 6 Colores: lila, morado...",Bic,30.0,Plumones y Marcadores
3176,SAM-S24 FE-8+256-GRIS DS,SMARTPHONE SAMSUNG S24 FE 5G 8+256 GB IMPORTAD...,SMARTPHONE SAMSUNG S24 FE 5G 8+256 GB IMPORTAD...,Samsung,1.0,Celulares y Tabletas


## Normlaiza texto:

In [5]:

# --- Normalización igual que en entrenamiento ---
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

# Uso Red

# Nuevas Categorias

In [6]:
# Crear el DataFrame
nuevas_categorias = pd.DataFrame({
    'id_subcategoria': list(range(1, 54)),  # IDs del 1 al 53
    'nombre_subcategoria': [
        "Celulares y Tabletas",
        "Accesorios para Computadores",
        "Componentes Informáticos",
        "Periféricos",
        "Computadores",
        "Cables de Red",
        "Monitores",
        "Puntos de Venta",
        "Protección de Poder",
        "Almacenamiento",
        "Electrodomésticos",
        "Audio y Video",
        "Software y Garantias",
        "Equipamiento de Redes y Conectividad",
        "Videovigilancia",
        "Control de acceso",
        "Alarmas, Detección e Iluminación",
        "Canalización",
        "Energía y componentes eléctricos",
        "Localización y rastreo",
        "Automatización y domótica",
        "Accesorios y herramientas",
        "Impresoras",
        "Impresoras 3D",
        "Multifuncional",
        "Plotter, Maquina de Coser e Impresión Laser",
        "Consumible",
        "Repuestos y Accesorios",
        "Bolígrafos",
        "Plumones y Marcadores",
        "Borradores",
        "Lápiz y Lápiz de Color",
        "Sujetadores",
        "Folder, Sobres y Carpetas",
        "Notas Adhesivas e Índice",
        "Grapadoras",
        "Adhesivos/Cintas",
        "Fundas, Mochilas y Maletines",
        "Accesorios de Oficina",
        "Corte",
        "Papel",
        "Corrector y Cinta Correctora",
        "Calculadora",
        "Sacapuntas",
        "Lienzos, Oleos, Pinturas y Tintas",
        "Casa y Jardín",
        "Scanner",
        "Libretas y Blocks",
        "Videojuegos",
        "Organizadores",
        "Sellos / Foliadores / Etiquetadores",
        "Material Didáctico / Manualidades",
        "Ropa"
    ]
})

# Mostrar el DataFrame
print(nuevas_categorias)

    id_subcategoria                          nombre_subcategoria
0                 1                         Celulares y Tabletas
1                 2                 Accesorios para Computadores
2                 3                     Componentes Informáticos
3                 4                                  Periféricos
4                 5                                 Computadores
5                 6                                Cables de Red
6                 7                                    Monitores
7                 8                              Puntos de Venta
8                 9                          Protección de Poder
9                10                               Almacenamiento
10               11                            Electrodomésticos
11               12                                Audio y Video
12               13                         Software y Garantias
13               14         Equipamiento de Redes y Conectividad
14               15      

# Cargar Modelo

In [8]:
# Cargar el modelo guardado
model = tf.keras.models.load_model(r"C:\Users\MoisesEugenioNavaMar\red_neuronal\Aplicativo-Categorizacion\modelo_categorias (2).keras")

# Ver datos

In [10]:
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- Normalización igual que en entrenamiento ---
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

# Cargar la base original para reconstruir tokenizer y LabelEncoder
df_datos = df_productos.copy()
# Normalizar columnas
for col in ['cnombre', 'cdescripcion', 'cmarca']:
    df_datos[col] = df_datos[col].fillna('').astype(str).apply(normalize_text)

# Separar etiquetados
df_labeled = df_datos[df_datos['subcategoria'].notna()].copy()
df_labeled['texto'] = df_labeled['cnombre'] + ' ' + df_labeled['cdescripcion'] + ' ' + df_labeled['cmarca']

# Reconstruir tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_labeled['texto'])

# Reconstruir LabelEncoder
le = LabelEncoder()
le.fit(df_labeled['subcategoria'])


LabelEncoder()

In [11]:
import pickle

# Guardar
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

with open("labelencoder.pkl", "wb") as f:
    pickle.dump(le, f)

# FUncion

In [12]:
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from unicodedata import normalize
import re


In [13]:
def normalize_text(text):
    text = text.lower()
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

In [14]:
# Cargar modelo
model = tf.keras.models.load_model(r"C:\Users\MoisesEugenioNavaMar\red_neuronal\Aplicativo-Categorizacion\modelo_categorias (2).keras")

In [15]:
# Cargar Tokenizer y LabelEncoder
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open("labelencoder.pkl", "rb") as f:
    le = pickle.load(f)


In [16]:
# Ejemplo: predecir categoría para nuevo texto
nuevo_texto = ["Laptop HP 14 pulgadas procesador intel core i5"]

In [17]:

def normalize_text(text):
    text = text.lower()
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

In [18]:
# Normalizar y convertir a secuencia
texto_norm = [normalize_text(t) for t in nuevo_texto]
seq = tokenizer.texts_to_sequences(texto_norm)
X_new = pad_sequences(seq, maxlen=200)

In [19]:
# Predecir
y_pred = model.predict(X_new)
categoria_pred = le.inverse_transform(np.argmax(y_pred, axis=1))
print(categoria_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
['Computadores']


In [20]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76682 entries, 0 to 76681
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   csku             76682 non-null  object 
 1   cnombre          76682 non-null  object 
 2   cdescripcion     76682 non-null  object 
 3   cmarca           76674 non-null  object 
 4   id_subcategoria  59834 non-null  float64
 5   subcategoria     59834 non-null  object 
dtypes: float64(1), object(5)
memory usage: 3.5+ MB
